# Imports

In [ ]:
import numpy as np
import pandas as pd

import copy
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import math
import datetime
import scipy as sp
import utm

# For a bigger output image
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# Subsampling time-intervals:

In [ ]:
def subsample_obs_time(df_in, time_interval_sec, obs_per_interval):

    obs = obs_per_interval
    time_inteval = time_interval_sec

    #Max and min time
    maxtime = max(df_in['datetimestamp'].values)
    mintime = min(df_in['datetimestamp'].values)

    #Total number of time sections in the data
    total_iter = int(maxtime/time_inteval) +1

    
    #start interval, inclusive
    lowerbound = mintime 
    #initial size, exclusive
    upperbound = time_inteval 

    #Result DATAFRAME
    df_sub_result = df_in.copy(deep = True)
    df_sub_result = df_sub_result.iloc[0:0] #drop all entries in the dataframe

    for subset_walk in np.arange(1,total_iter+2,1):#each subset present
        boolean_selection = ((lowerbound < df_in['datetimestamp']) & (df_in['datetimestamp'] <= upperbound)).values

        #subset for this interval
        time_subset = copy.deepcopy(df_in.iloc[boolean_selection,:])

        # subsampling the time sibset
        if obs < len(time_subset): #if the no of observations is lessthat the total in the timestep continue, else nothing
            for subsample in np.arange(0,len(time_subset), obs): #step with size = obs to sample each value in the set
                df_sub_result = df_sub_result.append(time_subset.iloc[subsample,:])

        #update bounds
        #start interval, inclusive
        lowerbound = copy.deepcopy(upperbound)
        upperbound = time_inteval*subset_walk

    return df_sub_result

## Other functions needed

In [ ]:
def dataframe_stats (df_in):
    print("Length:",len(df_in['datetimestamp']))
    print("Total time:",max(df_in['datetimestamp']))
    print("Min Long:",min(df_in['long']))
    print("Max Long:",max(df_in['long']))
    print("Min lat:",min(df_in['lat']))
    print("Max lat:",max(df_in['lat']))

# Kalman - Conversions:

In [ ]:
def convert_cog(data_in):
    return (data_in)*np.pi/180 #convert to radians

def convert_rot(data_in):
    return (data_in/60)*np.pi/180 #rad per second
    
def convert_sog(data_in):
    return data_in*0.5144444444 #meter per second

def convert_dataset(data_in, time_interval_sub, each_obs_sub):
    #Converting time date:
    df_sorted = copy.deepcopy(data_in.sort_values(by = 'datetimestamp', ascending = True ))
    #Getting the date of the first observation
    first_obs_date = df_sorted.iloc[0,6]

    #Subtracting the dates to get the seconds from start to finish
    df_sorted['datetimestamp'] = ((pd.to_datetime(df_sorted['datetimestamp']) - pd.to_datetime(first_obs_date)).dt.total_seconds())

    #Sub sample time interval in seconds, after dates has been converted to startin at 0 sec to 5000s (example)
    test = subsample_obs_time(df_sorted, time_interval_sub, each_obs_sub)
    df_sorted = copy.deepcopy(test)
    

    #Conversion of Course over ground
    df_sorted['cog'] = convert_cog(df_sorted['cog'])
    
    #Convert Speed over ground to mps
    df_sorted['sog'] = convert_sog(df_sorted['sog'])
    
    #Convert rate of turn 
    df_sorted['rot'] = convert_rot(df_sorted['rot'])
    
    #Making the order same as testset    
    df_sorted = df_sorted[['datetimestamp', 'lat', 'long', 'cog', 'sog','rot']]
    
    # Caluclating the the cartesian coordinates
    X = []
    Y = []
    
    #initial cartesian
    init_utm = utm.from_latlon(df_sorted['lat'].values[0], df_sorted['long'].values[0])
    X.append(init_utm[0])   #adding X coordinate
    Y.append(init_utm[1])   #Adding Y coordinate

    delta_time = [] #calculating delta time
    delta_time.append(0)
    for i in np.arange(1, len(df_sorted['datetimestamp']),1):
        delta_time.append(int(df_sorted['datetimestamp'].values[i]) - int(df_sorted['datetimestamp'].values[i-1]))
        #Caclulcating coordinates
        ans = utm.from_latlon(df_sorted['lat'].values[i], df_sorted['long'].values[i])
        #Adding the cartesian coordinates per time step
        X.append(copy.deepcopy(ans[0]))
        Y.append(copy.deepcopy(ans[1]))
    
   
    #Adding the delta time
    df_sorted.insert(1,'deltatime',delta_time) 
    #Adding cartesian coordiantes
    df_sorted['lat'] = Y
    df_sorted['long'] = X
   
    return df_sorted   

## Functions for the Kalman Filter

In [ ]:
def get_mat_A(deltaTime):
    A = np.identity(4)
    A[0,2] = deltaTime
    A[1,3] = deltaTime
    return A

def predict_state(A, X_prev, a_x, a_y, deltaTIME):
        
    Bu_k = np.array([[a_x*deltaTIME*deltaTIME/2],
                     [a_y*deltaTIME*deltaTIME/2],
                     [a_x*deltaTIME],
                     [a_y*deltaTIME]])
    
    pred_state = A.dot(X_prev) + Bu_k
    return pred_state

def pred_err_cov(A, Pk, Q):
    return A.dot(Pk).dot(np.transpose(A)) + Q

def Kalman_gain(P_k, H, R):
    above = P_k.dot(np.transpose(H))
    bottom = np.linalg.inv(H.dot(P_k).dot(np.transpose(H)) + R)
    return above.dot(bottom)

def adjust_pred_state(pred_state, Kk,z_k,H):
    #Where z_k == the real observed value to adjust for the errors made
    return pred_state + Kk.dot(z_k - H.dot(pred_state))

def update_err_cov(I,Kk,H,P_k):
    return (I - Kk.dot(H)).dot(P_k)    

def calc_a_xy(arr_in, arr_prev):
    
    sog_now = np.around(arr_in[5],8)
    sog_prev = np.around(arr_prev[5],8)
    
    cog_now = np.around(arr_in[4],8)
    cog_prev = np.around(arr_prev[4],8)
    
    rot_now = np.around(arr_in[6],6)    
    delta_time = arr_in[1]   
    a_x = (sog_now*np.cos(cog_now + rot_now*delta_time) - sog_prev*np.cos(cog_prev))/delta_time    
    a_y = (sog_now*np.sin(cog_now + rot_now*delta_time) - sog_prev*np.sin(cog_prev))/delta_time 
    
    return a_x, a_y


def convert_state(arr_in):
    sog = arr_in[5]
    cog = arr_in[4]    
    x = arr_in[2]
    y = arr_in[3]        
    Vx = sog*np.cos(cog)
    Vy = sog*np.sin(cog)
    
    return_arr = np.array([[x],[y],[Vx],[Vy]])
    
    return return_arr

def extract_long_lat_covariance(mat_in):
    Var1 = mat_in[0,0]
    Var2 = mat_in[1,1]
    Cov12 = mat_in[0,1]
    
    result = np.array([[Var1,Cov12],[Cov12,Var2]])
    return result

def set_R(R_in, arr_x, arr_y, arr_Vx, arr_Vy):
    var_x = np.var(arr_x)
    var_y = np.var(arr_y)
    var_Vx = np.var(arr_Vx)
    var_Vy = np.var(arr_Vy)
    
    R_in[0,0] = var_x
    R_in[1,1] = var_y
    R_in[2,2] = var_Vx
    R_in[3,3] = var_Vy
    
    return R_in
    
    

In [ ]:
test = np.arange(0,10)
test

In [ ]:
k = 5
test[(k-5):k]

# The Kalman Filter:

In [ ]:
ax_arr = []
ay_arr= []
Pk_save_states = []
def myKalman(pd_final, R,Q,P_k,show_plots = True):
    global ax_arr, ay_arr, df_final,Pk_save_states
    
    H = np.identity(4)
    I = np.identity(4)   

    # Kalman Initial state
    X_state = np.array([[0],[0],[0],[0]])
    
    # Lists to append data and values to, saving ttansitions:
    ax_arr = []
    ay_arr = []
    X_save_states = []
    Pk_save_states = []
    Pk_save_states.append(copy.deepcopy(P_k))

    data_counter = 1 # counter for the original data set obsevations

    max_time = int(max(pd_final.iloc[:,0]))
    min_time = int(min(pd_final.iloc[:,0]))

    data_counter_stop = max_time-min_time

    
    realSpeedX = []
    realSpeedY = []
    realSpdCount = []
    for i in np.arange(0,max_time,1):
        

        '''If we observe the value we observe this'''
        if(i == 0 or (int(pd_final.iloc[data_counter,0]) == i and data_counter < data_counter_stop)):
            zk_prev = pd_final.iloc[data_counter-1,:].values #previous observation
            z_k = pd_final.iloc[data_counter,:].values # convert to numpy array -observed value

            delta_TIME = copy.deepcopy(z_k[1]) #difference in time
            A = get_mat_A(delta_TIME)

            #Acceleration
            ax, ay = calc_a_xy(z_k,zk_prev) 
            ax_arr.append(ax)
            ay_arr.append(ay)

            z_k = copy.deepcopy(convert_state(z_k)) #convert state
            realSpdCount.append(i)
            realSpeedX.append(z_k[2][0])
            realSpeedY.append(z_k[3][0])
            X_state = copy.deepcopy(convert_state(zk_prev))

            #Kalman Predict--------------------------------------------------------------------------------------------
            X_state = copy.deepcopy(predict_state(A, X_state, ax, ay, delta_TIME))
            P_k =  copy.deepcopy(pred_err_cov(A, P_k, Q))

            #Kalman Measuremenet update--------------------------------------------------------------------------------
            Kalman_k = copy.deepcopy(Kalman_gain(P_k, H, R))
            X_state = copy.deepcopy(adjust_pred_state(X_state, Kalman_k, z_k, H))
            P_k = copy.deepcopy(update_err_cov(I, Kalman_k, H, P_k))


            #increment counter
            data_counter += 1

            #Saving states
            X_save_states.append(copy.deepcopy(X_state))
            Pk_save_states.append(copy.deepcopy(P_k))

        else:
            '''This code will be excecuted when we have no observation'''
            ay = 0
            ax = 0
            A = get_mat_A(1)
            #Kalman Predict --------------------------------------------------------------------------
            X_state = copy.deepcopy(predict_state(A, X_state, ax, ay, 1))
            P_k =  copy.deepcopy(pred_err_cov(A,P_k,Q))

            #Saving states
            Pk_save_states.append(copy.deepcopy(P_k))
            X_save_states.append(copy.deepcopy(X_state))




    # Saving the coordinates:
    X_coord = []
    Y_coord = []
    Pk_coord  = []
    SpeedX = []
    SpeedY = []
    
    
    

    for i in range(0,len(X_save_states)):
        if i > 1 and  i % 1 == 0:
            X_coord.append(X_save_states[i][0][0])
            Y_coord.append(X_save_states[i][1][0])
            SpeedX.append(X_save_states[i][2][0]) #Speed in X axis
            SpeedY.append(X_save_states[i][3][0]) #Speed in Y axis
            
            #Pk_coord.append(extract_long_lat_covariance(Pk_save_states[i]))

    #Extracting the original data:
    X_orig = copy.deepcopy(pd_final.iloc[:,2])
    Y_orig = copy.deepcopy(pd_final.iloc[:,3])

    
    if(show_plots):
        #Graph outputs:
        
        x_y_size = 14
        title_size = 17
        
        '''Speed in the X-axis'''
        plt.figure(figsize = (10,5))
        plt.plot(np.arange(0,len(SpeedX),1), SpeedX, '-', color = 'royalblue',linewidth = 1)
        plt.plot(realSpdCount, realSpeedX,'o-', color = 'red', markersize = 2)
        plt.title("Latitudinal Speed Over Ground - DKF", size =  title_size)
        legend_data = ["DKF Estimation","Observed SOG"]
        plt.legend(legend_data,loc = "lower center",fontsize =  x_y_size)
        plt.xlabel("Time (s)", size = x_y_size)
        plt.ylabel("Speed over ground (m/s)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        
        plt.show()
        

        '''Speed in the Y-axis'''
        plt.figure(figsize = (10,5))
        plt.plot(np.arange(0,len(SpeedY),1),SpeedY, '-',  color = 'royalblue',linewidth = 1)
        plt.plot(realSpdCount, realSpeedY,'o-', color = 'red', markersize = 2)
        legend_data = ["DKF Estimation","Observed SOG"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.title("Longitudinal Speed Over Ground - DKF", size =  title_size)
        plt.xlabel("Observation number", size = x_y_size)
        plt.ylabel("Speed over ground (m/s))", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()

        '''X coordinate tracking'''
        plt.plot(realSpdCount,Y_orig[0:(len(realSpdCount))], "ro")
        plt.plot(np.arange(0,len(Y_coord),1),Y_coord, "b.")
        legend_data = ["Original Observation","DKF Estiamte"]
        plt.legend(legend_data,loc = 'upper center', fontsize = x_y_size)
        plt.title("Latitudinal coordinate tracking: $\hat{x} $ vs $x$", size =  title_size)
        plt.xlabel("Observation Number", size = x_y_size)
        plt.ylabel("Latitude (UTM)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()

        '''Y coordinate tracking'''
        plt.plot(realSpdCount,X_orig[0:(len(realSpdCount))], "ro")
        plt.plot(np.arange(0,len(X_coord),1),X_coord, "b.")
        plt.title("Longitudinal coordinate tracking: $\hat{y} $ vs $y$", size =  title_size)
        legend_data = ["Original Observation","DKF Estimate"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.xlabel("Observation Number", size = x_y_size)
        plt.ylabel("Longitude (UTM)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()
        
        '''XY coordinate tracking'''
        plt.figure(figsize = (9,4.5))
        plt.plot(Y_orig[0:(len(realSpdCount))],X_orig[0:(len(realSpdCount))],'o-', color = 'red', markersize = 3)
        plt.plot(Y_coord,X_coord, '-',  color = 'royalblue',linewidth = 3)
        plt.title("Predicted Trajectory - DKF", size = title_size)
        legend_data = ["Original Trajectory","DKF Estimate"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.xlabel("Latitude (UTM)", size = x_y_size)
        plt.ylabel("Longitude (UTM)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()
   
    
    return np.array(X_coord),np.array(Y_coord),X_orig,Y_orig,ax_arr,ay_arr, SpeedX, SpeedY, np.array(realSpdCount)


# Importing dataset & getting linear trajectory:

In [ ]:
pd_dataset = None
import os
print(os.getcwd())
pd_dataset = pd.read_csv(r"304927000.csv")

#Boundaries sampling from and to
lower = 50
upper = 750
x_y_size = 14
title_size = 17
plt.plot(pd_dataset['long'].values[lower:upper],pd_dataset['lat'].values[lower:upper],'b.')
plt.title("Trajectory")
plt.show()

plt.figure(figsize = (9,5))

mean_speed = np.mean(pd_dataset['sog'].values[lower:upper])
median_speed = np.median(pd_dataset['sog'].values[lower:upper])
plt.axhline(y = mean_speed, linestyle = "--", alpha = 0.85, color = "darkgreen" )
plt.axhline(y = median_speed, linestyle = "--", alpha = 0.85, color = "darkorange" )
plt.plot(np.arange(0,len(pd_dataset['sog'].values[lower:upper]),1),pd_dataset['sog'].values[lower:upper],'o-', color = 'r',alpha = 0.95,markersize = 3.8, linewidth = 1)

legend_data = [str("Average Speed "+ str(np.round(mean_speed,2))+" kts"), 
               str("Median Speed "+ str(np.round(median_speed,2))+" kts"),"Observed Speed"]
plt.legend(legend_data,loc = 'best', fontsize = 12)

plt.title("Observed SOG for Each Observation",size = title_size)
plt.xlabel("Observation Number", size = x_y_size)
plt.ylabel("Speed Over Ground (knots)",size = x_y_size)
plt.yticks(size = x_y_size)
plt.xticks(size = x_y_size)
plt.show()

# SUB SAMPLING & TRAJECTORY EXTRACTION

In [ ]:
lower = 300
upper = 750
myrange = np.arange(lower,upper,1) #to cut the part thats goint to be used

pd_dataset_new = convert_dataset(pd_dataset.iloc[myrange,:].reset_index(drop = True),240,1)
#The pandas dataframe index needs to be resetted so that the indexing does'nt play a factor
df_sampled = copy.deepcopy(pd_dataset_new)

### Trajectory statistics:

In [ ]:
#Getting some stats on the extracted trajectory
dataframe_stats(pd_dataset_new)    

# Setting up $P_k$

In [ ]:
#IMO Standards
var_X = 10
var_Y = 10
var_spdX = 0.3
var_spdY = 0.3

P_k = np.array([[var_X],[var_Y],[var_spdX],[var_spdY]]).dot(np.transpose(
      np.array([[var_X],[var_Y],[var_spdX],[var_spdY]])))

# Setting up $Q$:



In [ ]:
# IMO standards:
var_X = 10
var_Y = 10
var_spdX = 0.3
var_spdY = 0.3

Q = np.array([[var_X],[var_Y],[var_spdX],[var_spdY]]).dot(np.transpose(
      np.array([[var_X],[var_Y],[var_spdX],[var_spdY]]))) 

# Setting up R:


In [ ]:
R = np.diag(np.diag(P_k))

# Running the Kalman filter

In [ ]:
X_samp_pred, Y_samp_pred, X_samp_orig, Y_samp_orig, _, _, _, _, Kalman_observed_sec = myKalman(df_sampled, R,Q,P_k,True)

In [ ]:
df_sampled

# Linear Implementation

### This will be a function of time
We will be estimating X Y SpeedOverGround CourseOverGround. With this we can calculate the speed in the X and Y axis respectively.

#### Idea:
$X_{new} =  X_{old} + Speed_{x-axis} \times time$

$Y_{new} =  Y_{old} + Speed_{y-axis} \times time$


# Import the linear regression model from Scikit learn

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
def AIS_linear(df_in, lin_remember_history, show_plots = True):
    '''
        df_in:  Pandas Data Frame,
                AIS data frame with more or less linear trajectory
        lin_remember_history: integer,  (default = 5)
                This integer value should specify the number of historical observations the linear method 
                are based on
    '''
    #REMEMBER HISTORY OF MODEL
    remember_history = lin_remember_history 
    
    # Extracting the values from the dataframe
    X_have = df_in['long'].values
    Y_have = df_in['lat'].values
    time = df_in['datetimestamp'].values.astype(int) #exctracting time as an array
    #Extracting speed
    speed = df_in['sog'].values
    course = df_in['cog'].values
      
    #Intitial parameters
    current_course = course[0] #initial course value. It remains constant only update if a new value comes
    
    #Time range in seconds
    sec_range = np.arange(0,int(max(time))+1,1)
    
    sec_observed = [] # to rememember when a second was an observed second

    
    #To save historical times
    time_history = np.array([]) # thislist will keep track of the last times recorded, intitial time == 0
    speed_history = np.array([]) # this list keep track of the speeds recorded
    pred_speed = np.array([]) # this list stores the predicted values

    #Speed in X Y
    pred_speedX =  []
    pred_speedY = []
    orig_speedX = []
    orig_speedY = []

    #Saving prediction
    speed_pred = []

    # Saving the X and Y predictions
    X_loc_pred = []
    Y_loc_pred = []
    
    # Setting initial values
    X_new =  X_have[0]
    Y_new = Y_have[0]
    X_error = 0
    Y_error = 0


    #Initial Regression guess with a speed of 4.8, just a random speed value nothing special about it
    reg = LinearRegression()
    reg.fit(np.array([0]).reshape(-1,1),np.array([4.8]).reshape(-1,1))

    #counting the entries to appropriatly measure difference
    counter_sec = 0 
    for sec in sec_range:    
        '''IF WE OBSERVE the value'''

        if(time[counter_sec] == sec):
            sec_observed.append(sec) #We observed index at this second

            if len(time_history) > remember_history : #if the list is long enough
                speed_history = np.delete(speed_history,0) #remove 1st entry in array
                time_history = np.delete(time_history,0) # remove 1st entry in array

            #Saving the last moments up to specified history amount    
            time_history = np.append(time_history,sec)
            speed_history = np.append(speed_history,speed[counter_sec])
            current_course =  course[counter_sec]

            #Fit a new model
            reg = LinearRegression()
            reg.fit(time_history.reshape(-1,1),speed_history.reshape(-1,1))

            #PREDICTING SPEED
            predict_speed_value = reg.predict(np.array([sec]).reshape(-1,1))
            pred_speed = np.append(pred_speed,predict_speed_value)

            #SPEED IN X AN Y AXIS
            pred_X_axis_speed = predict_speed_value[0][0]*np.cos(current_course)
            pred_speedX.append(pred_X_axis_speed)   
            pred_Y_axis_speed = predict_speed_value[0][0]*np.sin(current_course)
            pred_speedY.append(pred_Y_axis_speed)

            # SAVING ORIGINAL VALUES
            orig_speedX.append(speed[counter_sec]*np.cos(course[counter_sec]))
            orig_speedY.append(speed[counter_sec]*np.sin(course[counter_sec]))

            #PREDICTING MOVEMENT IN X AND Y
            X_new = X_new + pred_X_axis_speed
            X_error = X_have[counter_sec] - X_new  # to update next position to the correct spot c
            X_loc_pred.append(X_new)

            Y_new = Y_new + pred_Y_axis_speed
            Y_error = Y_have[counter_sec] - Y_new  # to update next position to the correct spot c
            Y_loc_pred.append(Y_new)

            #Counter to walk through list
            counter_sec += 1   
        else:
            #JUST PERDICT
            predict_speed_value = reg.predict(np.array([sec]).reshape(-1,1))
            #SPEED
            pred_speed = np.append(pred_speed, predict_speed_value)
            #SPEED IN X AN Y AXIS 
            pred_X_axis_speed = predict_speed_value[0][0]*np.cos(current_course)
            pred_speedX.append(pred_X_axis_speed)   
            pred_Y_axis_speed = predict_speed_value[0][0]*np.sin(current_course)
            pred_speedY.append(pred_Y_axis_speed)

            #PREDICTING MOVEMENT IN X AND Y
            X_new = X_new + pred_X_axis_speed + X_error
            X_error = 0 #reset the error after the correction

            Y_new = Y_new + pred_Y_axis_speed + Y_error
            Y_error = 0 #reset the error after the correction
            
            Y_loc_pred.append(Y_new)
            X_loc_pred.append(X_new)


    
    if(show_plots):
        # PLOTTING - Functions
#         plt.plot(sec_range,pred_speed,'r-')
#         plt.plot(time,speed,"b-")
#         legend_data = ["Linear tracking", "Original data"]
#         plt.legend(legend_data,loc = 'best', fontsize = 16)
#         plt.title("Linear speed tracking vs true speed", size = 20)
#         plt.ylabel("Speed over ground (m/s)", size = 20)
#         plt.xlabel("Time (s)", size = 20)
#         plt.show()


        x_y_size = 12
        title_size = 16
        plt.figure(figsize = (10,5))
        
        
        plt.plot(time,np.array(orig_speedX),'o-', color = 'red', markersize = 2)
        plt.plot(sec_range,np.array(pred_speedX),'-', color = 'royalblue',linewidth = 1)
        
        legend_data = ["Linear Tracking", "Original Data"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.title("Latitudinal Speed Over Ground - LRM", size =  title_size)
        plt.ylabel("Speed over ground (m/s)", size = x_y_size)
        plt.xlabel("Time (s)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)      
        plt.show()

        plt.figure(figsize = (10,5))
        plt.plot(sec_range,np.array(pred_speedY),'o-', color = 'red', markersize = 2)
        plt.plot(time,np.array(orig_speedY),'-', color = 'royalblue',linewidth = 1)
        legend_data = ["Linear Tracking", "Original Data"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.title("Longitudinal Speed Over Ground - LRM", size =  title_size)
        plt.xlabel("Time (s)", size = x_y_size)
        plt.ylabel("Speed over ground (m/s)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()

        plt.plot(sec_range,np.array(X_loc_pred),'bo')
        plt.plot(time,X_have,'r.')
        legend_data = ["Linear Method", "Original Data"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.title("Latitudinal coordinate tracking: $\hat{x} $ vs $x$", size = title_size)
        plt.ylabel("Latitude (UTM)", size = x_y_size)
        plt.xlabel("Observation Number", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()

        plt.plot(sec_range,np.array(Y_loc_pred),'bo')
        plt.plot(time,Y_have,'r.')
        legend_data = ["LRM Estimate", "Original Observation"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.title("Longitudinal coordinate tracking: $\hat{y} $ vs $y$", size = title_size)
        plt.ylabel("Longitude (UTM)", size = x_y_size)
        plt.xlabel("Observation Number", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()
        
        plt.figure(figsize = (9,4.5))
        
        plt.plot(X_have, Y_have,'o-', color = 'red', markersize = 3)
        plt.plot(X_loc_pred,Y_loc_pred, '-',  color = 'royalblue',linewidth = 3)
        
        legend_data = ["Original Trajectory","LRM Estimate"]
        plt.legend(legend_data,loc = 'best', fontsize = x_y_size)
        plt.title("Predicted Trajectory - LRM", size = title_size)
        plt.ylabel("Longitude (UTM)", size = x_y_size)
        plt.xlabel("Latitude (UTM)", size = x_y_size)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.show()   

   
    return np.array(X_loc_pred), np.array(Y_loc_pred), np.array(pred_speed), np.array(sec_observed)

In [ ]:
X_samp_pred, Y_samp_pred, X_samp_orig, Y_samp_orig, _, _, _, _, Kalman_observed_sec = myKalman(df_sampled, R,Q,P_k,True)
pred_linear_X, pred_linear_Y,pred_linear_SOG,linear_sec_ob = AIS_linear(df_origMMSI,5,True)

In [ ]:
def calculate_errors(df_orig,df_sub, pred_X, pred_Y, observed_sec,myTitle, show_plots = False ):
    time_steps = df_orig['datetimestamp'].values.astype(int)   
    
    x_y_size = 14
    title_size = 16
    max_time = max(observed_sec)
    time_steps = time_steps[time_steps <= max_time] 
    
    
    X_pred_orig = pred_X[time_steps]
    Y_pred_orig = pred_Y[time_steps]  
    
    test = np.arange(0,len(X_pred_orig),1)
    

    X_orig = df_orig['long'].values[test]  
    Y_orig = df_orig['lat'].values [test] 
    

    
    X_MSE = np.sum((X_orig - X_pred_orig )**2)/len(test)
    Y_MSE = np.sum((Y_orig - Y_pred_orig )**2)/len(test)
    
    Euclid_AVG = np.sum(np.sqrt((X_orig - X_pred_orig)**2 + (Y_orig - Y_pred_orig )**2))/len(X_pred_orig)
    
    
    X_error = X_orig - X_pred_orig
    Y_error = Y_orig - Y_pred_orig 
    Euclid_err = np.sqrt((X_orig - X_pred_orig)**2 + (Y_orig - Y_pred_orig )**2)
    
    if(show_plots):
        plt.figure(figsize = (10,5))
        plt.plot(time_steps,X_error,'.')
        plt.plot(time_steps,Y_error,'.')
        plt.plot(time_steps,Euclid_err,'.')
        plt.title(myTitle, size = title_size)
        plt.xlabel("Time (s)", size = x_y_size)
        plt.ylabel("Error (m)", size = x_y_size)
        legend_lab = ["Latitudinal","Longitudinal","Euclidean Distance"]
        plt.ylim(-500,500)
        plt.yticks(size = x_y_size)
        plt.xticks(size = x_y_size)
        plt.legend(legend_lab, loc = "lower center", title = "Errors", ncol = 3, 
                   fontsize = x_y_size, title_fontsize = 14.5)
        plt.show()
    
    
        print("X_MSE",X_MSE)
        print("Y_MSE",Y_MSE)
        print("Euclid_AVG",Euclid_AVG) 
    
    return X_MSE, Y_MSE, Euclid_AVG,Euclid_err

## Dataset

In [ ]:
pd_dataset = None 
pd_dataset = pd.read_csv(r"304927000.csv")
lower = 300
upper = 750
myrange = np.arange(lower,upper,1) #to cut the part thats goint to be used
df_origMMSI = convert_dataset(pd_dataset.iloc[myrange,:].reset_index(drop = True),240,1)

In [ ]:
df_mmsi_and_range = pd.DataFrame(columns=['MMSI','START', 'END'])

arr_MMSI = np.array(['538004506.csv',
                       '226105000.csv',
                       '235097013.csv',
                       '305714000.csv',
                       '304519000.csv',
                       '304927000.csv',
                       '636092331.csv',
                       '224389000.csv',
                       '220540000.csv',
                       '211286440.csv',
                       '207138000.csv',
                       '207138000.csv',
                       '215901000.csv',
                       '227146400.csv',
                       '314237000.csv',
                       '565494000.csv',
                       '244740921.csv',
                       '227330000.csv',
                       '227372000.csv',
                       '228130000.csv',
                       '228727000.csv',
                       '227988000.csv',
                       '244925000.csv',
                       '247224200.csv',
                       '249104000.csv',
                       '518866000.csv',
                       '276700000.csv',
                       '577228000.csv',
                       '227558000.csv',
                       '227364000.csv'])
# print(np.array(['../NARI MMSI CSVs/']),arr_MMSI)


MMSI_starts = np.array([10 ,420,200,5  ,300,300,4358,128,120,2210,150,790 ,50 ,960 ,1000,20 ,5200,
               50 ,10 ,20 ,100,1600,0  ,210,200,350,300,300,350,495])
MMSI_ends   = np.array([350,850,400,600,750,750,4850,375,560,2800,590,1200,600,1500,1500,550,5600,
               550,420,450,500,2000,450,620,650,800,700,750,800,850])


In [ ]:
arr_MMSI

# Kalman Filter & Linear Funcition

In [ ]:
my_sequence = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])
df_results = pd.DataFrame(columns=['MMSI',
                                   'Method', 
                                   'Sub1',
                                    'Sub2',
                                    'Sub3',
                                    'Sub4',
                                    'Sub5',
                                    'Sub6',
                                    'Sub7',
                                    'Sub8',
                                    'Sub9',
                                    'Sub10',
                                    'Sub11',
                                    'Sub12',
                                    'Sub13',
                                    'Sub14',
                                    'Sub15',
                                    'Sub16',
                                    'Sub17',
                                    'Sub18',
                                    'Sub19',
                                    'Sub20',
                                    'Sub21'])

my_sub_sec = np.arange(240,1800,60)

for sub_sec_count in my_sub_sec:
    print("\n\n\n MY SUBSAMPLE LENGTH:", sub_sec_count, "\n\n\n")

    for data_set in range(0,len(arr_MMSI)):
        my_MMSI = str("./NARI MMSI CSVs/"+arr_MMSI[data_set])
        start = MMSI_starts[data_set] 
        end = MMSI_ends[data_set]

        print("\n\n\nMMSI NO:",my_MMSI,"START:", start, "END:", end)

        pd_dataset = pd.read_csv(my_MMSI)
        lower = start
        upper = end
        myrange = np.arange(lower,upper,1) #to cut the part thats goint to be used
        df_origMMSI = convert_dataset(pd_dataset.iloc[myrange,:].reset_index(drop = True),240,1)


        Kalman_error_EuclidXY =   []
        Linear_error_EuclidYX =   []

        for sub_sample_data in my_sequence:
            #print("\n\n\n\n\n\nSUB SAMPLE SIZE: ",sub_sample_data )
            df_subMMSI = convert_dataset(pd_dataset.iloc[myrange,:].reset_index(drop = True),sub_sec_count,sub_sample_data)

            Kalman_X_pred, Kalman_Y_pred, X_samp_orig, Y_samp_orig, _, _, _,_,Kalman_observed_sec = myKalman(df_subMMSI, R,Q,P_k, False)
            #print('------------------------------------- LINEAR METHOD ---------------------------------------')
            pred_linear_X, pred_linear_Y,pred_linear_SOG,linear_sec_ob = AIS_linear(df_subMMSI,5,False)

            #print('------------------------------------- KALMAN ERROR ---------------------------------------')
            Kalman_X_MSE, Kalman_Y_MSE, Kalman_Euclid_AVG, _ = calculate_errors(df_origMMSI,df_subMMSI,Kalman_Y_pred,Kalman_X_pred,Kalman_observed_sec, False)
            #print('------------------------------------- LINEAR ERROR ---------------------------------------')
            Linear_X_MSE, Linear_Y_MSE, Linear_Euclid_AVG,_ = calculate_errors(df_origMMSI,df_subMMSI,pred_linear_X,pred_linear_Y,linear_sec_ob, False)

            #APPENDINGS

            Kalman_error_EuclidXY.append(Kalman_Euclid_AVG)
            Linear_error_EuclidYX.append(Linear_Euclid_AVG)

        Kalman_error_EuclidXY = np.array(Kalman_error_EuclidXY)
        Linear_error_EuclidYX = np.array(Linear_error_EuclidYX)

        df_results = df_results.append({'MMSI': my_MMSI,'Method': 'Kalman', 
                                        'Sub1':Kalman_error_EuclidXY[0],
                                        'Sub2':Kalman_error_EuclidXY[1],
                                        'Sub3':Kalman_error_EuclidXY[2],
                                        'Sub4':Kalman_error_EuclidXY[3],
                                        'Sub5':Kalman_error_EuclidXY[4],
                                        'Sub6':Kalman_error_EuclidXY[5],
                                        'Sub7':Kalman_error_EuclidXY[6],
                                        'Sub8':Kalman_error_EuclidXY[7],
                                        'Sub9':Kalman_error_EuclidXY[8],
                                        'Sub10':Kalman_error_EuclidXY[9],
                                        'Sub11':Kalman_error_EuclidXY[10],
                                        'Sub12':Kalman_error_EuclidXY[11],
                                        'Sub13':Kalman_error_EuclidXY[12],
                                        'Sub14':Kalman_error_EuclidXY[13],
                                        'Sub15':Kalman_error_EuclidXY[14],
                                        'Sub16':Kalman_error_EuclidXY[15],
                                        'Sub17':Kalman_error_EuclidXY[16],
                                        'Sub18':Kalman_error_EuclidXY[17],
                                        'Sub19':Kalman_error_EuclidXY[18],
                                        'Sub20':Kalman_error_EuclidXY[19],
                                        'Sub21':Kalman_error_EuclidXY[20]}, ignore_index=True)


        df_results = df_results.append({'MMSI': my_MMSI,'Method': 'Linear', 
                                       'Sub1':Linear_error_EuclidYX[0],
                                        'Sub2':Linear_error_EuclidYX[1],
                                        'Sub3':Linear_error_EuclidYX[2],
                                        'Sub4':Linear_error_EuclidYX[3],
                                        'Sub5':Linear_error_EuclidYX[4],
                                        'Sub6':Linear_error_EuclidYX[5],
                                        'Sub7':Linear_error_EuclidYX[6],
                                        'Sub8':Linear_error_EuclidYX[7],
                                        'Sub9':Linear_error_EuclidYX[8],
                                        'Sub10':Linear_error_EuclidYX[9],
                                        'Sub11':Linear_error_EuclidYX[10],
                                        'Sub12':Linear_error_EuclidYX[11],
                                        'Sub13':Linear_error_EuclidYX[12],
                                        'Sub14':Linear_error_EuclidYX[13],
                                        'Sub15':Linear_error_EuclidYX[14],
                                        'Sub16':Linear_error_EuclidYX[15],
                                        'Sub17':Linear_error_EuclidYX[16],
                                        'Sub18':Linear_error_EuclidYX[17],
                                        'Sub19':Linear_error_EuclidYX[18],
                                        'Sub20':Linear_error_EuclidYX[19],
                                        'Sub21':Linear_error_EuclidYX[20],}, ignore_index=True)

    my_csv_string =  str("KalmanVSlinearResults_28Oct21_"+ str(sub_sec_count) +".csv")
    df_results.to_csv(my_csv_string, index = False, header=True)

In [ ]:
# print('Kalman')
# print(len(Kalman_X_pred))
# print(len(Kalman_Y_pred))
# print("-")
# print(Kalman_observed_sec)

# print('Linear')
# print(pred_linear_X)
# print(pred_linear_Y)
# print(pred_linear_SOG)
# print(linear_sec_ob)

In [ ]:
df_results

In [ ]:
df_results.to_csv(r"KalmanVSlinearResults_1_21_420.csv", index = False, header=True)

In [ ]:
my_stats = pd.read_csv(r"KalmanVSlinearResults_240.csv")


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(my_stats)

In [ ]:
my_stats

In [ ]:
Kalman_stats = my_stats.iloc[0,:]
Linear_stats = my_stats.iloc[1,:]

In [ ]:
Kalman_stats.head()

In [ ]:
Linear_stats.head()

In [ ]:
# my_stats.to_csv("All_stats.csv")

In [ ]:
my_csv = np.arange(300,1860,60)

df_DKF_vs_LRM =  pd.read_csv(r"KalmanVSlinearResults_240.csv")
for csv in my_csv:
    my_csv_string = None
    my_csv_string =  str("./Kalman TimeIntervals/KalmanVSlinearResults_"+ str(csv) +".csv")
    temp_dataset = pd.read_csv(my_csv_string)
    df_DKF_vs_LRM = df_DKF_vs_LRM.append(temp_dataset)

In [ ]:

df_DKF_vs_LRM

In [ ]:
Kalman_stats = df_DKF_vs_LRM.iloc[np.arange(0,df_DKF_vs_LRM.shape[0],2),:]
Linear_stats = df_DKF_vs_LRM.iloc[np.arange(1,df_DKF_vs_LRM.shape[0],2),:]

In [ ]:
Kalman_stats
Linear_stats

In [ ]:
Kal_sub1 = np.average(Kalman_stats['Sub1'])
Kal_sub2 = np.average(Kalman_stats['Sub2'])
Kal_sub3 = np.average(Kalman_stats['Sub3'])
Kal_sub4 = np.average(Kalman_stats['Sub4'])
Kal_sub5 = np.average(Kalman_stats['Sub5'])
Kal_sub6 = np.average(Kalman_stats['Sub6'])
Kal_sub7 = np.average(Kalman_stats['Sub7'])
Kal_sub8 = np.average(Kalman_stats['Sub8'])
Kal_sub9 = np.average(Kalman_stats['Sub9'])
Kal_sub10 = np.average(Kalman_stats['Sub10'])
Kal_sub11 = np.average(Kalman_stats['Sub11'])
Kal_sub12 = np.average(Kalman_stats['Sub12'])
Kal_sub13 = np.average(Kalman_stats['Sub13'])
Kal_sub14 = np.average(Kalman_stats['Sub14'])
Kal_sub15 = np.average(Kalman_stats['Sub15'])
Kal_sub16 = np.average(Kalman_stats['Sub16'])
Kal_sub17 = np.average(Kalman_stats['Sub17'])
Kal_sub18 = np.average(Kalman_stats['Sub18'])
Kal_sub19 = np.average(Kalman_stats['Sub19'])
Kal_sub20 = np.average(Kalman_stats['Sub20'])
Kal_sub21 = np.average(Kalman_stats['Sub21'])


Kalman_errors = np.array([Kal_sub1,
                 Kal_sub2,
                 Kal_sub3,
                 Kal_sub4,
                 Kal_sub5,
                 Kal_sub6,
                 Kal_sub7,
                 Kal_sub8,
                 Kal_sub9,                 
                 Kal_sub10,
                 Kal_sub11,
                 Kal_sub12,
                 Kal_sub13,
                 Kal_sub14,
                 Kal_sub15,
                 Kal_sub16,
                 Kal_sub17,
                 Kal_sub18,
                 Kal_sub19,
                 Kal_sub20,
                 Kal_sub21])

Kalman_errors_STDs= np.array([
                    np.std(Kalman_stats['Sub1']),
                    np.std(Kalman_stats['Sub2']),
                    np.std(Kalman_stats['Sub3']),
                    np.std(Kalman_stats['Sub4']),
                    np.std(Kalman_stats['Sub5']),
                    np.std(Kalman_stats['Sub6']),
                    np.std(Kalman_stats['Sub7']),
                    np.std(Kalman_stats['Sub8']),
                    np.std(Kalman_stats['Sub9']),
                    np.std(Kalman_stats['Sub10']),
                    np.std(Kalman_stats['Sub11']),
                    np.std(Kalman_stats['Sub12']),
                    np.std(Kalman_stats['Sub13']),
                    np.std(Kalman_stats['Sub14']),
                    np.std(Kalman_stats['Sub15']),
                    np.std(Kalman_stats['Sub16']),
                    np.std(Kalman_stats['Sub17']),
                    np.std(Kalman_stats['Sub18']),
                    np.std(Kalman_stats['Sub19']),
                    np.std(Kalman_stats['Sub20']),
                    np.std(Kalman_stats['Sub21'])                   ])

Kalman_errors_UP = Kalman_errors +Kalman_errors_STDs
Kalman_errors_DOWN = Kalman_errors-Kalman_errors_STDs


Linear_sub1 = np.average(Linear_stats['Sub1'])
Linear_sub2 = np.average(Linear_stats['Sub2'])
Linear_sub3 = np.average(Linear_stats['Sub3'])
Linear_sub4 = np.average(Linear_stats['Sub4'])
Linear_sub5 = np.average(Linear_stats['Sub5'])
Linear_sub6 = np.average(Linear_stats['Sub6'])
Linear_sub7 = np.average(Linear_stats['Sub7'])
Linear_sub8 = np.average(Linear_stats['Sub8'])
Linear_sub9 = np.average(Linear_stats['Sub9'])
Linear_sub10 = np.average(Linear_stats['Sub10'])
Linear_sub11 = np.average(Linear_stats['Sub11'])
Linear_sub12 = np.average(Linear_stats['Sub12'])
Linear_sub13 = np.average(Linear_stats['Sub13'])
Linear_sub14 = np.average(Linear_stats['Sub14'])
Linear_sub15 = np.average(Linear_stats['Sub15'])
Linear_sub16 = np.average(Linear_stats['Sub16'])
Linear_sub17 = np.average(Linear_stats['Sub17'])
Linear_sub18 = np.average(Linear_stats['Sub18'])
Linear_sub19 = np.average(Linear_stats['Sub19'])
Linear_sub20 = np.average(Linear_stats['Sub20'])
Linear_sub21 = np.average(Linear_stats['Sub21'])

Linear_errors = np.array([Linear_sub1,
                 Linear_sub2,
                 Linear_sub3,
                 Linear_sub4,
                 Linear_sub5,
                 Linear_sub6,
                 Linear_sub7,
                 Linear_sub8,
                 Linear_sub9,
                 Linear_sub10,
                 Linear_sub11,
                 Linear_sub12,
                 Linear_sub13,
                 Linear_sub14,
                 Linear_sub15,
                 Linear_sub16,
                 Linear_sub17,
                 Linear_sub18,
                 Linear_sub19,
                 Linear_sub20,
                 Linear_sub21])

Linear_errors_STDs=  np.array([
                    np.std(Linear_stats['Sub1']),
                    np.std(Linear_stats['Sub2']),
                    np.std(Linear_stats['Sub3']),
                    np.std(Linear_stats['Sub4']),
                    np.std(Linear_stats['Sub5']),
                    np.std(Linear_stats['Sub6']),
                    np.std(Linear_stats['Sub7']),
                    np.std(Linear_stats['Sub8']),
                    np.std(Linear_stats['Sub9']),
                    np.std(Linear_stats['Sub10']),
                    np.std(Linear_stats['Sub11']),
                    np.std(Linear_stats['Sub12']),
                    np.std(Linear_stats['Sub13']),
                    np.std(Linear_stats['Sub14']),
                    np.std(Linear_stats['Sub15']),
                    np.std(Linear_stats['Sub16']),
                    np.std(Linear_stats['Sub17']),
                    np.std(Linear_stats['Sub18']),
                    np.std(Linear_stats['Sub19']),
                    np.std(Linear_stats['Sub20']),
                    np.std(Linear_stats['Sub21'])
                    ])




Linear_errors_UP = Linear_errors +Linear_errors_STDs
Linear_errors_DOWN =Linear_errors -Linear_errors_STDs


# We are merging all the datasets to get a better average

In [ ]:
len(Linear_stats)
len(Kalman_stats)

In [ ]:
df_DKF_vs_LRM

In [ ]:
Kalman_stats

In [ ]:
my_sequence = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])
mpl.rcParams['figure.figsize'] = (10,4)

linear_color = "#e76f51"
kalman_color = "#005f73"

plt.plot(my_sequence,Kalman_errors,"o-", color = kalman_color)
plt.plot(my_sequence,Linear_errors,"o-",color = linear_color)
plt.plot(my_sequence,Kalman_errors_UP,"--",color = kalman_color, alpha = 1)
plt.plot(my_sequence,Linear_errors_UP,"--",color = linear_color, alpha = 1)
plt.plot(my_sequence,Kalman_errors_DOWN,"--",color = kalman_color, alpha = 1)
plt.plot(my_sequence,Linear_errors_DOWN,"--",color = linear_color, alpha = 1)

plt.fill_between(my_sequence,Kalman_errors_UP,Kalman_errors_DOWN, facecolor = kalman_color,alpha = 0.3)
plt.fill_between(my_sequence,Linear_errors_UP,Linear_errors_DOWN, facecolor = linear_color,alpha = 0.3)

# plt.fill_between(my_sequence,Kalman_errors_UP,Linear_errors_DOWN, facecolor = kalman_color,alpha = 0.9)

plt.xlabel("Under sampling rates ($\lambda_{s_{i}}$)", fontsize = 14)
plt.ylabel('Mean Euclidean Error (m)', fontsize = 14)
plt.title('DKF and LRM prediction errors', fontsize = 18)

plt.xticks(np.arange(1, 22, step=1))
plt.yticks(size = 12)
plt.xticks(size = 12)

plt.xlim(0.9,5)
plt.ylim(-50,300)
#ax.set_xlabel('time [s]', fontsize='large', fontweight='bold')

legend_data = ['DKF','LRM', 'DKF + $ \sigma$', "LRM + $\sigma$" ]
plt.legend(legend_data, title = "Models & SD",title_fontsize = 16,loc = 'upper left', fontsize = 12, ncol = 2)
plt.show()

In [ ]:
myTest = np.array([10,20,30,40,50,60,70])

for i in myTest:
    string =  str(str(i) +".csv")
    print(string)

In [ ]:
np.arange(480,1860,60)

In [ ]:
df_sampled.to_csv(string)

In [ ]:
#Deepcopy the new dataset to the sampled one
df_sampled = copy.deepcopy(pd_dataset_new)

plt.plot(df_sampled['long'],df_sampled['lat'],'b.')
plt.title("Extracted Linear Trajectory - MMSI:304927000 ", size = 20)
plt.xlabel("X coordinate (UTM)", size = 20)
plt.ylabel("Y coordinate (UTM)", size = 20)
plt.yticks(size = 18)
plt.xticks(size = 18)
plt.show()


plt.plot(np.arange(0,len(df_sampled['sog']),1),df_sampled['sog'],'m.')
plt.title("Speed over ground per observation - MMSI:304927000", size = 20)
plt.xlabel("Observation from extracted trajectory", size = 20)
plt.ylabel("Speed over ground (SOG)", size = 20)
plt.yticks(size = 18)
plt.xticks(size = 18)
plt.show()

In [ ]:
304927000
arr_MMSI[5]

In [ ]:
my_sequence = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

# my_index = 5

# my_MMSI = arr_MMSI[my_index]
# start = MMSI_starts[my_index] 
# end = MMSI_ends[my_index] 

# print("\n\n\nMMSI NO:",my_MMSI,"START:", start, "END:", end)

# pd_dataset = df_sampled
# lower = start
# upper = end
# myrange = np.arange(lower,upper,1) #to cut the part thats goint to be used
# df_origMMSI = convert_dataset(pd_dataset.iloc[myrange,:].reset_index(drop = True),240,1)
df_origMMSI = df_sampled

Kalman_error_EuclidXY =   []
Linear_error_EuclidYX =   []

sub_sec_count = 300

for sub_sample_data in my_sequence:

    #print("\n\n\n\n\n\nSUB SAMPLE SIZE: ",sub_sample_data )
    df_subMMSI = convert_dataset(pd_dataset.iloc[myrange,:].reset_index(drop = True),sub_sec_count,sub_sample_data)

    Kalman_X_pred, Kalman_Y_pred, X_samp_orig, Y_samp_orig, _, _, _,_,Kalman_observed_sec = myKalman(df_subMMSI, R,Q,P_k, True)
    #print('------------------------------------- LINEAR METHOD ---------------------------------------')
    pred_linear_X, pred_linear_Y,pred_linear_SOG,linear_sec_ob = AIS_linear(df_subMMSI,5,True)
    

    #print('------------------------------------- KALMAN ERROR ---------------------------------------')
    Kalman_X_MSE, Kalman_Y_MSE, Kalman_Euclid_AVG = calculate_errors(df_origMMSI,df_subMMSI,Kalman_Y_pred,
                                                                     Kalman_X_pred,Kalman_observed_sec, 
                                                                     "DKF errors per observed observation"
                                                                     ,True)
    
    #print('------------------------------------- LINEAR ERROR ---------------------------------------')
    Linear_X_MSE, Linear_Y_MSE, Linear_Euclid_AVG = calculate_errors(df_origMMSI,df_subMMSI,pred_linear_X,pred_linear_Y,
                                                                     linear_sec_ob, 
                                                                     "LRM errors per observed observation",
                                                                     True)

    #APPENDINGS

    Kalman_error_EuclidXY.append(Kalman_Euclid_AVG)
    Linear_error_EuclidYX.append(Linear_Euclid_AVG)

Kalman_error_EuclidXY = np.array(Kalman_error_EuclidXY)
Linear_error_EuclidYX = np.array(Linear_error_EuclidYX)

df_results = df_results.append({'MMSI': my_MMSI,'Method': 'Kalman', 
                                'Sub1':Kalman_error_EuclidXY[0],
                                'Sub2':Kalman_error_EuclidXY[1],
                                'Sub3':Kalman_error_EuclidXY[2],
                                'Sub4':Kalman_error_EuclidXY[3],
                                'Sub5':Kalman_error_EuclidXY[4],
                                'Sub6':Kalman_error_EuclidXY[5],
                                'Sub7':Kalman_error_EuclidXY[6],
                                'Sub8':Kalman_error_EuclidXY[7],
                                'Sub9':Kalman_error_EuclidXY[8],
                                'Sub10':Kalman_error_EuclidXY[9],
                                'Sub11':Kalman_error_EuclidXY[10],
                                'Sub12':Kalman_error_EuclidXY[11],
                                'Sub13':Kalman_error_EuclidXY[12],
                                'Sub14':Kalman_error_EuclidXY[13],
                                'Sub15':Kalman_error_EuclidXY[14],
                                'Sub16':Kalman_error_EuclidXY[15],
                                'Sub17':Kalman_error_EuclidXY[16],
                                'Sub18':Kalman_error_EuclidXY[17],
                                'Sub19':Kalman_error_EuclidXY[18]}, ignore_index=True)

df_results = df_results.append({'MMSI': my_MMSI,'Method': 'Linear', 
                               'Sub1':Linear_error_EuclidYX[0],
                                'Sub2':Linear_error_EuclidYX[1],
                                'Sub3':Linear_error_EuclidYX[2],
                                'Sub4':Linear_error_EuclidYX[3],
                                'Sub5':Linear_error_EuclidYX[4],
                                'Sub6':Linear_error_EuclidYX[5],
                                'Sub7':Linear_error_EuclidYX[6],
                                'Sub8':Linear_error_EuclidYX[7],
                                'Sub9':Linear_error_EuclidYX[8],
                                'Sub10':Linear_error_EuclidYX[9],
                                'Sub11':Linear_error_EuclidYX[10],
                                'Sub12':Linear_error_EuclidYX[11],
                                'Sub13':Linear_error_EuclidYX[12],
                                'Sub14':Linear_error_EuclidYX[13],
                                'Sub15':Linear_error_EuclidYX[14],
                                'Sub16':Linear_error_EuclidYX[15],
                                'Sub17':Linear_error_EuclidYX[16],
                                'Sub18':Linear_error_EuclidYX[17],
                                'Sub19':Linear_error_EuclidYX[18]}, ignore_index=True)

# my_csv_string =  str("Results_FOR_"+ my_MMSI + str(sub_sec_count) +".csv")
# df_results.to_csv(my_csv_string, index = False, header=True)

In [ ]:
OG_DATA = pd.read_csv(r"304927000.csv")

MMSI_test_original = convert_dataset(OG_DATA.iloc[lower:upper,:].reset_index(drop = True),240,1)
SUBSET = convert_dataset(OG_DATA.iloc[lower:upper,:].reset_index(drop = True),240,1)
print(lower,upper)

In [ ]:
X_samp_pred, Y_samp_pred, X_samp_orig, Y_samp_orig, _, _, _, _, Kalman_observed_sec = myKalman(SUBSET, R,Q,P_k,True)

In [ ]:
pred_linear_X, pred_linear_Y,pred_linear_SOG,linear_sec_ob = AIS_linear(SUBSET,3,True)

In [ ]:
df_orig,df_sub, pred_X, pred_Y, observed_sec,myTitle, show_plots = False ):

In [ ]:
kalman_ERROR = []
for i in np.arange(1,17):
    kalman_each_interval = []
    for interval in [240,300,360,420,480,540,600,660,720,780,840,900,960,1020]:

        sub_sample = i
        MMSI_test_original = convert_dataset(OG_DATA.iloc[lower:upper,:].reset_index(drop = True),interval,sub_sample)
        SUBSET = convert_dataset(OG_DATA.iloc[lower:upper,:].reset_index(drop = True),interval,sub_sample)


        X_samp_pred, Y_samp_pred, X_samp_orig, Y_samp_orig, _, _, _, _, Kalman_observed_sec = myKalman(SUBSET, R,Q,P_k,False)
        Kalman_X_MSE, Kalman_Y_MSE, Kalman_Euclid_AVG, Kalman_EUC = calculate_errors(MMSI_test_original,SUBSET,Y_samp_pred, X_samp_pred,Kalman_observed_sec,"DKF Trajectory Prediction Error per observation", False)
        kalman_each_interval.append(Kalman_Euclid_AVG)
        
    kalman_ERROR.append(np.mean(kalman_each_interval))

In [ ]:
np.mean(kalman_ERROR)
#138.1035

In [ ]:
print(Kalman_Euclid_AVG,)

In [ ]:
pred_linear_X, pred_linear_Y,pred_linear_SOG,linear_sec_ob = AIS_linear(SUBSET,3,True)

In [ ]:
LRM_ERROR = []
for i in np.arange(1,17):
    LRM_each_interval = []
    for interval in [240,300,360,420,480,540,600,660,720,780,840,900,960,1020]:
        sub_sample = i
        MMSI_test_original = convert_dataset(OG_DATA.iloc[lower:upper,:].reset_index(drop = True),interval,sub_sample)
        SUBSET = convert_dataset(OG_DATA.iloc[lower:upper,:].reset_index(drop = True),interval,sub_sample)
        
        pred_linear_X, pred_linear_Y,pred_linear_SOG,linear_sec_ob = AIS_linear(SUBSET,3,False)
        Linear_X_MSE, Linear_Y_MSE, Linear_Euclid_AVG, Linear_EUCLID = calculate_errors(MMSI_test_original,SUBSET,pred_linear_X,pred_linear_Y,
                                                                             linear_sec_ob, 
                                                                         "LRM Trajectory Prediction Error per observation",
                                                                         False)
        LRM_each_interval.append(np.mean(Linear_Euclid_AVG))
    LRM_ERROR.append(np.mean(LRM_each_interval))


In [ ]:
np.mean(LRM_ERROR)
#242.98

In [ ]:
df_DKF_vs_LRM =  pd.read_csv(r"KalmanVSlinearResults_240.csv")
df_DKF_vs_LRM
